# Problem Statement

El proyecto trata de descubrir clientes potenciales para una distribuidora de material industrial.

En esta etapa:

El objetivo es: Mapeo de Productos

Se tienen los siguientes datos crudos:
* reportes de ventas anualizadas de 2020 a 2023


---

## Mapeo de Productos

In [18]:
import pandas as pd

In [19]:
VentasRaw = pd.read_excel("Ventas.xlsx")
VentasRaw.head(1)

,Código,Nombre,Cantidad,Unidad,Neto,Descuento,DescuentoNeto,Impuesto,Total,Year,UserRfc,ProveedorID,ProveedorName
0,11184,LIGA #18,135,LIB,10462.5,0,10462.5,1674,12136.5,2020,GUOR86061511A,P043,ALMA ANGELINA PANTOJA GOMEZ


In [20]:
ComprasRaw = pd.read_excel("Compras.xlsx")
ComprasRaw.head(1)

,Código,Nombre,Cantidad,Unidad,Neto,Descuento,DescuentoNeto,Impuesto,Total,Year,UserRfc,ProveedorID,ProveedorName
0,11184,LIGA #18,135,LIB,10462.5,0,10462.5,1674,12136.5,2020,GUOR86061511A,P043,ALMA ANGELINA PANTOJA GOMEZ


Para cruzar los datos solo se necesitan las siguientes variables:
* Codigo
* nombre
* Unidad

In [21]:
Rows = ["Código","Nombre","Unidad"]

In [22]:
ComprasData = pd.DataFrame()
for i in Rows:
    ComprasData[i] = ComprasRaw[i]

In [23]:
ComprasData.head(1)

,Código,Nombre,Unidad
0,11184,LIGA #18,LIB


In [24]:
VentasData = pd.DataFrame()
for i in Rows:
    VentasData[i] = VentasRaw[i]

In [25]:
VentasData.head(1)

,Código,Nombre,Unidad
0,11184,LIGA #18,LIB
